In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00


In [ ]:
import transformers
import torch

In [ ]:
!pip install -U langchain-community

In [ ]:
! huggingface-cli login

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
import os

In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
  url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
  file_path = "the-verdict.txt"
  urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

In [ ]:
document = raw_text

In [ ]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
chunk = text_spliter.split_text(document)

In [ ]:
len(chunk)

116

3 Creating DB

In [ ]:
!pip install chromadb

In [ ]:
persist_directory = "db"
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectordb = Chroma.from_texts(chunk, embedding, persist_directory=persist_directory)

In [ ]:
retriver = vectordb.as_retriever()

In [ ]:
!pip install llama-cpp-python

## Install required libraries

### Subtask:
Install necessary libraries like `llama-cpp-python`.


**Reasoning**:
The subtask is to install the `llama-cpp-python` library. The provided code cell already does this using `pip install`. I will execute this cell to complete the subtask.



In [ ]:
!pip install llama-cpp-python

## Download llama2 model

### Subtask:
Download a suitable Llama2 model file in a compatible format (e.g., GGML or GGUF).


**Reasoning**:
I need to download a Llama2 model file in a compatible format (GGUF) to the current working directory. I will use `wget` for this purpose and download a specific Llama-2-7b-chat-hf GGUF model from Hugging Face.



In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf -O llama-2-7b-chat.Q4_K_M.gguf

## Initialize llamacpp

### Subtask:
Use the downloaded model file to initialize the `LlamaCpp` class.


**Reasoning**:
The previous attempt to initialize LlamaCpp failed due to a KeyError related to 'model_path'. The instructions explicitly state to set the `model_path` parameter. Therefore, I need to correct the initialization by providing the correct parameter name and the path to the downloaded model file.



In [ ]:
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.4,
    max_tokens=512,
    n_ctx=2048,
    n_gpu_layers=20,
    verbose=True,
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    chain_type="stuff"
)

In [ ]:
# Step 5: Ask questions
while True:
    query = input("\n🧠 Ask something about the document (or type 'exit'): ")
    if query.lower() == "exit":
        break
    answer = qa_chain.run(query)
    print(f"\n🤖 Answer: {answer}")


🧠 Ask something about the document (or type 'exit'): who is jack


/tmp/ipython-input-82-3403371536.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)
Llama.generate: 1 prefix-match hit, remaining 288 prompt tokens to eval
llama_perf_context_print:        load time =    6295.90 ms
llama_perf_context_print: prompt eval time =   89779.95 ms /   288 tokens (  311.74 ms per token,     3.21 tokens per second)
llama_perf_context_print:        eval time =    5974.77 ms /     9 runs   (  663.86 ms per token,     1.51 tokens per second)
llama_perf_context_print:       total time =   95764.12 ms /   297 tokens



🤖 Answer:  Jack is the protagonist of this passage.

🧠 Ask something about the document (or type 'exit'): summarize the story


Llama.generate: 45 prefix-match hit, remaining 197 prompt tokens to eval
llama_perf_context_print:        load time =    6295.90 ms
llama_perf_context_print: prompt eval time =   62611.51 ms /   197 tokens (  317.82 ms per token,     3.15 tokens per second)
llama_perf_context_print:        eval time =   15418.81 ms /    25 runs   (  616.75 ms per token,     1.62 tokens per second)
llama_perf_context_print:       total time =   78055.00 ms /   222 tokens



🤖 Answer:  I don't know.
Unhelpful Answer: Jack gave up painting because he married his interest in interesting women.

🧠 Ask something about the document (or type 'exit'): exit


**Reasoning**:
Define a simple query and use the initialized llm object to get a response from the model.

